# Comptre rendu projet Maths/Info
### Clement Desroches & Mathis Bourdin

    
L'objectif de ce projet est de rédiger un code Python permettant de calculer les lignes de niveau d'une fonction de deux variables réelles et à valeurs réelles. Les fonctions seront supposées continûment diffrentiables sur les domaines de définition.

Afin de réaliser ce projet nous aurons besoin de plusieurs biblihotèques. Nous commencons donc par les importer :

    - Numpy : Pour ces fonctions mathématiques et sa classe nparray
    - Autograd et sa partie numpy : Pour determiner le gradient des fonctions étudiées.
    - Numpy.linalg : Pour sa fonction norm
    - Matplotlib.pyplot : Pour afficher les resultats

In [ ]:
import numpy as np
from autograd import numpy as np
import autograd
from numpy.linalg import norm
import matplotlib.pyplot as plt

Pour commencer ce projet, nous étudions dans un premier temps les fonctions sur le carré unité. Il est nécessaire de determiner un point de départ pour le calcul de la ligne de niveau. Nous commencons donc par ecrire une fonction find_seed retournant un point sur l'arête gauche sur ce carré qui appartient à la ligne de niveau recherchée. 

Pour déterminer ce point nous utilisons une dichotomie, un algorithme simple ayant peu de contraintes et ayant un convergence relativement rapide si on la compare à sa simplicité et à sa fiabilité. Nous condisèrerons dans ce projet que pour une fonction f, la ligne de niveau de valeur c à un point d'intersection avec l'arête gauche du carré unité si g(0,0) <= c <= g(0,1) ou g(0,0) >= c >= g(0,1). Si cette condtion n'est pas vérifiée nous renvoyons la valeur **None**.

Voici la fonction find_seed :

In [ ]:
def find_seed(g, c=0, eps=2**(-26)):
    #Conditions aren't verified.
    if not(g(0,0) <= c <= g(0,1) or g(0,0) >= c >= g(0,1)):
        return None
    
    a, b = 0, 1
    f = lambda x: g(0,x) - c
    while b-a > eps:
        d = (a+b)/2
        if f(d)*f(a) > 0:
            a = d
        else :
            b = d
    return (a+b)/2

Verifions que celle ci fonctionne :

In [ ]:
y = find_seed(lambda x, y: x**2 + y**2, c=0.5)
print(y**2)

Le résultat de find_seed est satisfaisant.

Maintenant que nous avons le point de départ de notre ligne de niveau, il faut determiner la suite de cette ligne.

Une méthode apparaissant immédiatement est de suivre un vecteur tangent à cette ligne de niveau. On sait que le gradient est toujours orthogonal aux lignes de niveau. Il suffit alors de déterminer le gradient à l'aide d'autograd, de prend l'orthogonal et de suivre ce vecteur. Aucun test sur le sens du vecteur n'est effectué, nous cherchons juste à voir si la méthode est viable ou non.

Le dernier point est retiré des listes est retiré car il n'appartient plus au carré unité.

Voici cette fonction (ici nommée simple_contour_naif) et un test effectué avec :

In [ ]:
def simple_contour_naif(f, c=0.0, delta=0.01):
    fc = lambda x,y: f(x,y) - c
    eps=2**(-26)
    def grad_f(x,y):
        g = autograd.grad
        return np.r_[g(fc,0)(x,y),g(fc,1)(x,y)]
    lx = [0.]
    ly = [find_seed(fc,0)]
    if ly[-1] == None:
        return [], []
    while 0<=lx[-1]<=1 and 0<=ly[-1]<=1:
        grad = grad_f(lx[-1],ly[-1])
        orth = np.array([grad[1],-1*grad[0]])
        grad *= delta/norm(grad)
        orth *= delta/norm(orth)

        lx.append(lx[-1]+orth[0])
        ly.append(ly[-1]+orth[1])
    return lx[:-1], ly[:-1]

In [ ]:
q,s = simple_contour_naif(lambda x,y: (x**2+y**2)**0.5, c = 0.5)
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(q[:],s[:])
phi = np.linspace(0,np.pi/2,201)
ax.plot([0.5*np.cos(p) for p in phi], [0.5*np.sin(p) for p in phi])

On remarque un ecart entre les deux courbes qui semble croissant avec le nombre de déplacement. ce résultat n'est pas surprenant. L'erreur n'etant pas corrigée, celle ci grandit au fur et à mesure des itérations.

Nous avons donc décidé de rédiger une nouvelle version de cette fonction. Celle ci effectue une correction de l'erreur à chaque itération à l'aide d'une dichotomie angulaire. On calcul de vecteur tangent de la meme manière en le normalisant au pas de déplacement. On cherche ensuite un nouveau vecteur déplacement en faisant varier son angle par rapport au vecteur initial avec cette dichotomie. Ainsi l'erreur est corrigée a chaque itération. Cela permet de réduire en théorie sa croissance.

Il existe peut être des test plus intelligents sur le sens du vecteur deplacement (par exemple un unique test en début de code) mais cela nous assure pleinement du bon sens de ce vecteur.

Voici cette fonction et un test effectué :

In [ ]:
def simple_contour(f, c=0.0, delta=0.01):
    fc = lambda x,y: f(x,y) - c
    eps=2**(-26)
    def grad_f(x,y):
        g = autograd.grad
        return np.r_[g(fc,0)(x,y),g(fc,1)(x,y)]
    lx = [0.]
    ly = [find_seed(fc,0)]
    if ly[-1] == None:
        return [], []
    while 0<=lx[-1]<=1 and 0<=ly[-1]<=1:
        grad = grad_f(lx[-1],ly[-1])
        orth = np.array([grad[1],-1*grad[0]])
        grad *= delta/norm(grad)
        orth *= delta/norm(orth)
        if len(lx)==1 and orth[0]<0:
            orth *= -1
        if len(lx)>1 and np.dot(orth,np.array([lx[-1]-lx[-2],ly[-1]-ly[-2]])) < 0:
            orth *= -1
        fdic = lambda phi: np.cos(phi)*grad + np.sin(phi)*orth
        fphi = lambda phi: fc(lx[-1]+fdic(phi)[0],ly[-1]+fdic(phi)[1])
        a, b, d = 0, np.pi, None
        while b-a > eps:
            d = (a+b)/2
            if fphi(a)*fphi(d) > 0:
                a = d
            else:
                b = d
        v = fdic(d)
        lx.append(lx[-1]+v[0])
        ly.append(ly[-1]+v[1])
    return lx[:-1], ly[:-1]


In [ ]:
q,s = simple_contour(lambda x,y: (x**2+y**2)**0.5, c = 0.5)
print(abs(q[-1]-0.5))
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(q[:],s[:])
phi = np.linspace(0,np.pi/2,201)
ax.plot([0.5*np.cos(p) for p in phi], [0.5*np.sin(p) for p in phi])

Les deux courbes sont confondues ici sur l'image et l'erreur en fin d'itération est d'environ 3e-05. Ce résultat est très acceptable et le code s'éxècute rapidement. Cette fonction semble donc une bonne réponse au problème posé.

La fonction contour est celle fournie sur le Discourse sans modification.

Le test final de ce projet est le même que founit en exemple du projet.

In [ ]:
# Rotators
# ------------------------------------------------------------------------------
LEFT, UP, RIGHT, DOWN = 0, 1, 2, 3  # clockwise


def rotate_direction(direction, n=1):
    return (direction + n) % 4


def rotate(x, y, n=1):
    if n == 0:
        return x, y
    elif n >= 1:
        return rotate(1 - y, x, n - 1)
    else:
        assert n < 0
        return rotate(x, y, n=-3 * n)


def rotate_function(f, n=1):
    def rotated_function(x, y):
        xr, yr = rotate(x, y, -n)
        return f(xr, yr)

    return rotated_function


# Complex Contouring
# ------------------------------------------------------------------------------

# Customize the simple_contour function used in contour :
# simple_contour = smart_simple_contour


def contour(f, c, xs=[0.0, 1.0], ys=[0.0, 1.0], delta=0.01):
    curves = []
    nx, ny = len(xs), len(ys)
    for i in range(nx - 1):
        for j in range(ny - 1):
            xmin, xmax = xs[i], xs[i + 1]
            ymin, ymax = ys[j], ys[j + 1]

            def f_cell(x, y):
                return f(xmin + (xmax - xmin) * x, ymin + (ymax - ymin) * y)

            done = set()
            for n in [0, 1, 2, 3]:
                if n not in done:
                    rotated_f_cell = rotate_function(f_cell, n)
                    x_curve_r, y_curve_r = simple_contour(rotated_f_cell, c, delta)
                    exit = None
                    if len(x_curve_r) >= 1:
                        xf, yf = x_curve_r[-1], y_curve_r[-1]
                        if xf == 0.0:
                            exit = LEFT
                        elif xf == 1.0:
                            exit = RIGHT
                        elif yf == 0.0:
                            exit = DOWN
                        elif yf == 1.0:
                            exit = UP
                    if exit is not None:  # a fully successful contour fragment
                        exit = rotate_direction(exit, n)
                        done.add(exit)

                    x_curve, y_curve = [], []
                    for x_r, y_r in zip(x_curve_r, y_curve_r):
                        x, y = rotate(x_r, y_r, n=-n)
                        x_curve.append(x)
                        y_curve.append(y)
                    x_curve = np.array(x_curve)
                    y_curve = np.array(y_curve)
                    curves.append(
                        (xmin + (xmax - xmin) * x_curve, ymin + (ymax - ymin) * y_curve))
    return curves

In [ ]:
def g(x,y):
    return np.exp(-x**2-y**2)

def l(x,y):
    return np.exp(-(x-1)**2-(y-1)**2)

def fonction(x,y):
    return 2*(g(x,y)-l(x,y))

niveaux = [-1.5,-1.,-0.5,0.,0.5,1.,1.5]
fig, ax = plt.subplots()

for i in niveaux:
    curves = contour(fonction,i,[-2.,-1.,0.,1.,2.,3.],[-1.,0.,1.,2.])
    for x,y in curves:
        ax.plot(x,y)


Le résultat obtenu semble sensiblement le même. Nous pouvons donc conclure que nos fonctions fournissent le résultat attendu. Le temps d'éxécution est également raisonnable (sur un coeur cadencé à 2.4 GHz).

Une piste d'amélioration de ce projet pourrait se faire sur la convergence de nos fonctions. Celle des dichotomie se fait légèrement ressentir. On pourrait remplacer celle ci par une méthode de Newton dans R2 à l'aide de la Jacobienne inversé.

Voici le rendu du projet dans un graph 3D.

In [ ]:
from numpy import linspace, meshgrid
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x_list, y_list = linspace(-3,3,201), linspace(-2,2,101)
X, Y = meshgrid(x_list, y_list)
ax.plot_surface(X, Y, fonction(X,Y),cmap='viridis')

niveaux = [-1.5,-1.,-0.5,0.,0.5,1.,1.5]

for i in niveaux:
    curves = contour(fonction,i,[-2.,-1.,0.,1.,2.,3.],[-1.,0.,1.,2.])
    for x,y in curves:
        ax.plot(x,y,i,color="red", linewidth = 2.5)

ax.set_title("Tracé de la fonction dans R2 avec lignes de niveau")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')